# Integrate with Amazon Bedrock Knowledge Bases:
After processed the audio and video files with a BDA project, next it is time to integrate with Bedrock KB.
## Steps involved in this integration: 
- Set up a knowledge base to parse documents using Amazon Bedrock Data Automation as the parser.
- Ingest the processed data into the knowledge base for retrieval and response generation.

<div class="alert alert-block alert-info">
<b>Note:</b> Please run this notebook after you finish running the first notebook: 01_data_prep_using_bda.ipynb, the notebook cell one at a time instead of using "Run All Cells" option.
</div>

# Setup notebook and boto3 clients

In this step, we will import some necessary libraries that will be used throughout this notebook. To use Amazon Bedrock Data Automation (BDA) with boto3, you'll need to ensure you have the latest version of the AWS SDK for Python (boto3) installed. Version Boto3 1.35.96 of later is required.

Note: At time of Public Preview launch, BDA is available in us-west-2 only.

In [ ]:
%pip install -r ../requirements.txt --no-deps --quiet

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import boto3
from botocore.exceptions import ClientError
from datetime import datetime
import time
from time import sleep
import pprint
import random
from tqdm import tqdm
from pathlib import Path
from IPython.display import JSON, IFrame, Audio, display, clear_output
import IPython.display as display
import logging
import sys
from pathlib import Path
import base64

# Get current path and go up two parent directories
current_path = Path().resolve()
parent_path = current_path.parent  # Go up two levels

# Add to sys.path if not already there
if str(parent_path) not in sys.path:
    sys.path.append(str(parent_path))

# Now you can import from utils
from utils.knowledge_base import BedrockKnowledgeBase

In [ ]:
#Clients
suffix = random.randrange(200, 900)

sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()["Account"]

region_name = "us-east-1" # can be removed ones BDA is GA and available in other regions.
region = region_name

s3_client = boto3.client('s3', region_name=region_name)

bda_client = boto3.client('bedrock-data-automation', region_name=region_name)
bda_runtime_client = boto3.client('bedrock-data-automation-runtime', region_name=region_name)

bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# Copy local BDA output files to a S3 bucket for KB integration
# Function to check if the bucket exists, if not, create the data_bucket
from utils.knowledge_base_operators import bucket_exists
suffix = random.randrange(200, 900)
bucket_name_kb = f'bedrock-bda-kb-{suffix}-1'            
# Create S3 bucket for the KB if it doesn't exist
if not bucket_exists(bucket_name_kb):
    print(f"Bucket '{bucket_name_kb}' does not exist. Creating it now...")
    if region == "us-east-1":
        s3_client.create_bucket(Bucket=bucket_name_kb)
    else:
        s3_client.create_bucket(
            Bucket=bucket_name_kb,
            CreateBucketConfiguration={'LocationConstraint': region}
        )
    print(f"Bucket '{bucket_name_kb}' created successfully.")
else:
    print(f"Bucket '{bucket_name_kb}' already exists.")


In [ ]:
import requests
import fitz
from PIL import Image
import io
import base64
import boto3
import os
from urllib.parse import urlparse

def download_pdf_from_url(url):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        return io.BytesIO(response.content)
    except requests.RequestException as e:
        print(f"Failed to download PDF from {url}: {e}")
        return None

def pdf_to_images(pdf_content, quality=75, max_size=(1024, 1024)):
    """Convert PDF to list of images"""
    images = []
    try:
        with fitz.open(stream=pdf_content.getvalue(), filetype="pdf") as doc:
            for page_num, page in enumerate(doc):
                # Get page pixmap
                pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))
                # Convert to PIL Image
                image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
                
                # Resize if needed
                if image.size[0] > max_size[0] or image.size[1] > max_size[1]:
                    image.thumbnail(max_size, Image.Resampling.LANCZOS)
                
                # Convert to bytes
                img_byte_arr = io.BytesIO()
                image.save(img_byte_arr, format='PNG', optimize=True, quality=quality)
                img_byte_arr.seek(0)
                
                images.append({
                    'page_num': page_num + 1,
                    'image_data': img_byte_arr
                })
        return images
    except Exception as e:
        print(f"Error converting PDF to images: {e}")
        return None

def upload_images_to_s3(images, bucket_name, s3_prefix, pdf_name):
    """Upload images to S3"""
    s3_client = boto3.client('s3')
    uploaded_files = []
    
    try:
        for img in images:
            # Create filename for the image
            base_name = os.path.splitext(pdf_name)[0]
            image_key = f"{s3_prefix}/{base_name}/page_{img['page_num']}.png"
            
            # Upload to S3
            s3_client.upload_fileobj(
                img['image_data'],
                bucket_name,
                image_key
            )
            
            uploaded_files.append(image_key)
            
        return uploaded_files
    except Exception as e:
        print(f"Error uploading to S3: {e}")
        return None

def process_pdf_to_images_s3(pdf_url, bucket_name, s3_prefix):
    """Main function to process PDF to images and upload to S3"""
    try:
        # Download PDF
        print(f"Downloading PDF from {pdf_url}")
        pdf_content = download_pdf_from_url(pdf_url)
        if not pdf_content:
            return {"error": "Failed to download PDF"}
        
        # Get PDF filename from URL
        pdf_name = os.path.basename(urlparse(pdf_url).path)
        
        # Convert to images
        print("Converting PDF to images")
        images = pdf_to_images(pdf_content)
        if not images:
            return {"error": "Failed to convert PDF to images"}
        
        # Upload to S3
        print("Uploading images to S3")
        uploaded_files = upload_images_to_s3(images, bucket_name, s3_prefix, pdf_name)
        if not uploaded_files:
            return {"error": "Failed to upload images to S3"}
        
        return {
            "status": "success",
            "pdf_name": pdf_name,
            "uploaded_files": uploaded_files,
            "total_pages": len(images)
        }
        
    except Exception as e:
        return {"error": str(e)}

# Usage
pdf_url = "https://s2.q4cdn.com/299287126/files/doc_financials/2024/q4/Webslides_Q424_Final.pdf"
bucket_name = bucket_name_kb  # Your bucket name
s3_prefix = "bda/dataset/pdf_images"  # Prefix for the images in S3

# Process PDF and upload images
result = process_pdf_to_images_s3(pdf_url, bucket_name, s3_prefix)
print(result)

# Verify uploads if successful
if result.get("status") == "success":
    s3_client = boto3.client('s3')
    for s3_key in result["uploaded_files"]:
        try:
            s3_client.head_object(Bucket=bucket_name, Key=s3_key)
            print(f"Verified: s3://{bucket_name}/{s3_key}")
        except Exception as e:
            print(f"Failed to verify {s3_key}: {str(e)}")


In [ ]:
# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"

knowledge_base_name = f"bedrock-multi-modal-kb-{suffix}"
knowledge_base_description = "Multi-modal RAG knowledge base."

foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

### Start the Knowledge Base creation 

In this notebook, the process of creating a KB is simplified by using a wrapper function from the knowledge_base.py file in "utils" folder of this notebook. The whole process of creating data source, creating a KB, creating an embedding index, saving the index in a vector data store is simplified by using this function. 


In [ ]:
## Please uncomment the data sources that you want to add and update the placeholder values accordingly.

#data=[{"type": "S3", "bucket_name": bucket_name, "inclusionPrefixes": ["bda/dataset/"]}]
data=[{"type": "S3", "bucket_name": bucket_name_kb}]


                # {"type": "SHAREPOINT", "tenantId": "888d0b57-69f1-4fb8-957f-e1f0bedf64de", "domain": "yourdomain",
                #   "authType": "OAUTH2_CLIENT_CREDENTIALS",
                #  "credentialsSecretArn": f"arn:aws::secretsmanager:{region_name}:secret:<<your_secret_name>>",
                #  "siteUrls": ["https://yourdomain.sharepoint.com/sites/mysite"]
                # },
    
                
pp = pprint.PrettyPrinter(indent=2)

### Step 1 - Create Knowledge Base with Multi modality

In [ ]:
# For multi-modal RAG While instantiating BedrockKnowledgeBase, pass multi_modal= True and choose the parser you want to use

knowledge_base = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name}',
    kb_description=knowledge_base_description,
    data_sources=data,
    multi_modal= True,
    parser= 'BEDROCK_DATA_AUTOMATION', #'BEDROCK_Data Automation service is used'
    chunking_strategy = "FIXED_SIZE", 
    suffix = f'{suffix}-f'
)

### Step 2 - Start data ingestion job to KB

Once the KB and data source(s) created, we can start the ingestion job for each data source. During the ingestion job, KB will fetch the documents from the data source, Parse the document to extract text, chunk it based on the chunking size provided, create embeddings of each chunk and then write it to the vector database, in this case OSS.

NOTE: Currently, you can only kick-off one ingestion job at one time.

In [ ]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base.start_ingestion_job()

In [ ]:
# keep the kb_id for invocation later in the invoke request
kb_id = knowledge_base.get_knowledge_base_id()
%store kb_id

### Step 3 -  Test the Knowledge Base
Now the Knowlegde Base is available we can test it out using the [**retrieve**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html) and [**retrieve_and_generate**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) functions. 

#### Testing Knowledge Base with Retrieve and Generate API

Let's first test the knowledge base using the retrieve and generate API. With this API, Bedrock takes care of retrieving the necessary references from the knowledge base and generating the final answer using a foundation model from Bedrock.

query = Give me the summary of the AWS Rethink podcast hosted by Nolan Chen and Malini Chatterjee?

The right response for this query is expected to fetch from a the audio transcript ingested in Knowledge Bases.

### Query image file

In [ ]:
query = "Show me figures of Amazon TTM operation income and net sales"

In [ ]:
foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"
# foundation_model = "amazon.nova-micro-v1:0"

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

### Show images

In [ ]:
import base64
import io
from PIL import Image
import matplotlib.pyplot as plt

def display_base64_image(base64_string, figsize=(15, 15)):
    try:
        # Remove the data:image/png;base64, prefix if present
        if 'data:image/png;base64,' in base64_string:
            base64_string = base64_string.split('base64,')[1]
            
        # Decode base64 string
        image_data = base64.b64decode(base64_string)
        
        # Convert to PIL Image
        image = Image.open(io.BytesIO(image_data))
        
        # Display the image
        plt.figure(figsize=figsize)
        plt.imshow(image)
        plt.axis('off')
        plt.show()
        
    except Exception as e:
        print(f"Error displaying image: {str(e)}")

# Extract and display the image from citations
citations = response['citations'][0]
if 'retrievedReferences' in citations:
    for ref in citations['retrievedReferences']:
        if 'content' in ref and 'byteContent' in ref['content']:
            base64_image = ref['content']['byteContent']
            print("Displaying citation image:")
            display_base64_image(base64_image)


### Clean up
Please make sure to uncomment and run the below section to delete all the resources.

In [ ]:
# delete role and policies
print("===============================Deleting Knowledge Base and associated resources==============================\n")
knowledge_base.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)

# Conclusion

By following this guide, you can effectively harness the power of Amazon Bedrock’s features to build a robust Multimodal RAG application tailored to your specific needs.